### setup

In [1]:
from book_to_table_func import *
import warnings
warnings.filterwarnings('ignore')
#from openai import OpenAI
from tqdm.auto import tqdm
from PyPDF2 import PdfReader
import ast
import os
import pandas as pd
import numpy as np

from book_to_graph_func import remove_quotes, ask_haike,chanks_to_df_for_graph,SYS_PROMPT
from book_to_table_func import book_to_chank_dict_list


### load doc and find headers

In [2]:
book = PdfReader(r"C:\Users\yotam\Desktop\books_to_table_llm\books\How_Not_to_Die.pdf")
#find headers:
full_book = ''
for i in range(9,438): #len(book.pages) #i cuted the starting pages and the notes in the end
        page = book.pages[i].extract_text().replace('\t',' ')#.replace('\n',' ')

        full_book = full_book + page + ' '

path = r"C:\Users\yotam\Desktop\books_to_table_llm\books\How_Not_to_Die.pdf"
headers = headers_scrape_How_Not_to_Die(path,jank='https://www.8freebooks.net',min_size=14.5)
headers_txt = [header[0] for header in headers]
headers_txt = headers_txt[3:320] #trash the part of the book that don't have content

### chank the doc by the headers

In [3]:
chank_dict,chank_list = book_to_chank_dict_list(full_book=full_book,headers_txt=headers_txt)

Salmonella67
Flaxseed163
Disease175
Citrus220
Coffee291


i need to make df with the answer of the llm with the data: node_1,node_2,edge,chunk_id,count

### put the chank in a nice df

#### one chank example:

In [18]:
result = ask_haike(user_prompt=f"context: ```{chank_list[21]}``` \n\n output: ",system_prompt=SYS_PROMPT,max_tokens=4000)

In [19]:
print(result.dict()['content'][0]['text'])

Here is the ontology of terms and their relations extracted from the given context:

[
   {
       "node_1": "Fiber",
       "node_2": "Bowel health",
       "edge": "Fiber has well-known effects on bowel health"
   },
   {
       "node_1": "Fiber",
       "node_2": "Cancer",
       "edge": "High fiber intake appears to reduce the risk of cancers of the colon and breast"
   },
   {
       "node_1": "Fiber",
       "node_2": "Diabetes",
       "edge": "High fiber intake appears to reduce the risk of diabetes"
   },
   {
       "node_1": "Fiber",
       "node_2": "Heart disease",
       "edge": "High fiber intake appears to reduce the risk of heart disease"
   },
   {
       "node_1": "Fiber",
       "node_2": "Obesity",
       "edge": "High fiber intake appears to reduce the risk of obesity"
   },
   {
       "node_1": "Fiber",
       "node_2": "Premature death",
       "edge": "High fiber intake appears to reduce the risk of premature death in general"
   },
   {
       "node_1": "Fibe

In [67]:
full_df,chanks_not_added=chanks_to_df_for_graph(chank_dict,prompt=SYS_PROMPT,max_token_per_chank=4000,verbos=True,test = False)

pass 10 chanks
eror in the api
ASTHMA
pass 20 chanks
pass 30 chanks
pass 40 chanks
pass 50 chanks
pass 60 chanks
eror in the JSON.LOADS
Salmonella
pass 70 chanks
pass 80 chanks
pass 90 chanks
eror in the JSON.LOADS
Isn’t Moderate Drinking Beneficial?
pass 100 chanks
pass 110 chanks
eror in the JSON.LOADS
Nitrosamines
pass 120 chanks
pass 130 chanks
eror in the JSON.LOADS
Coffee and Aspartame
pass 140 chanks
pass 150 chanks
pass 160 chanks
pass 170 chanks
pass 180 chanks
pass 190 chanks
eror in the JSON.LOADS
Black Currants and Bilberries
pass 200 chanks
eror in the JSON.LOADS
Exotic Fruits
eror in the JSON.LOADS
Too Much of a Good Thing?
pass 210 chanks
pass 220 chanks
eror in the JSON.LOADS
Even More Vegetables!
eror in the JSON.LOADS
Getting Kids (and Parents) to Eat Their Veggies
pass 230 chanks
pass 240 chanks
pass 250 chanks
eror in the JSON.LOADS
Is Liquid Smoke Safe?
pass 260 chanks
pass 270 chanks
eror in the JSON.LOADS
Let Me Help


In [69]:
full_df.to_csv('How_Not_to_Die_df_graph.csv', index=False)

In [81]:
len(chanks_not_added)
#12 chanks not added

12

when their is " in the text - the load_json not working, here a way to fix it:

In [106]:
print(remove_quotes(txt[txt.find('['):txt.find(']')+1]))

[
   {
       "node_1": "Salmonella",
       "node_2": "food poisoning",
       "edge": "Salmonella is the leading cause of food poisoning-related hospitalizations and deaths in the United States"
   },
   {
       "node_1": "Salmonella",
       "node_2": "eggs",
       "edge": "Many people associate Salmonella with eggs, and for good reason as Salmonella can survive in eggs cooked in various ways like scrambled, over-easy, and sunny-side-up"
   },
   {
       "node_1": "Salmonella",
       "node_2": "cooking methods",
       "edge": "The egg industry's own research found that Salmonella in eggs can survive scrambled, over-easy, and sunny-side-up cooking methods, with sunny-side-up being the riskiest"
   },
   {
       "node_1": "Salmonella",
       "node_2": "illness",
       "edge": "Salmonella infection typically causes fever, diarrhea, and severe abdominal cramps within 12 to 72 hours, and the illness can last 4 to 7 days, requiring hospitalization for children and the elderly"
   

In [3]:
df = pd.read_csv(r'C:\Users\yotam\Desktop\how_not_to_project\books_to_table_llm\df_saved\How_Not_to_Die_df_graph_v2.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2972 entries, 0 to 2971
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   node_1      2972 non-null   object
 1   node_2      2972 non-null   object
 2   edge        2972 non-null   object
 3   Chunk Name  2972 non-null   object
 4   count       2972 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 116.2+ KB


### node example:

In [19]:
df.iloc[99]

node_1                                         Secondhand smoke
node_2                                              Lung cancer
edge          Nonsmokers have a 20–30 percent higher risk of...
Chunk Name                                          LUNG CANCER
count                                                         4
Name: 99, dtype: object

: 

In [18]:

df.loc[99,'edge']


"Nonsmokers have a 20–30 percent higher risk of developing lung cancer if they're regularly exposed to cigarette smoke"